In [1]:
# library
import torch
import numpy as np
import pandas as pd
from torchmetrics import R2Score

r2score = R2Score()

torch.manual_seed(1)
np.random.seed(1)

In [2]:
# Model
class Net(torch.nn.Module):
  def __init__(self, s):
    super(Net, self).__init__()
    self.input   = torch.nn.Linear(4, s)
    self.output  = torch.nn.Linear(s, 1)

  def forward(self, x):
    z = torch.tanh(self.input(x))
    z = self.output(z)
    return z

In [3]:
# Model evaluation
def eval(model, testset):
    with torch.no_grad():
        pred_Y = model(testset.x_data)
    
    r2 = r2score(pred_Y, testset.y_data)
    return r2.item()

In [4]:
# Data class
class Data(torch.utils.data.Dataset):
  def __init__(self, src_file, start=None, end=None):
    df = pd.read_csv(src_file)
    Tca_k1   = np.array(df['ahu_supply_temp']).reshape(-1,1)[start+1: end+1]
    Tsa_k    = np.array(df['supply_discharge_temp']).reshape(-1,1)[start: end]
    Tsa_k1   = np.array(df['supply_discharge_temp']).reshape(-1,1)[start+1: end+1]
    valve_k2 = np.array(df['htg_valve_position']).reshape(-1,1)[start+2: end+2]

    tmp_x = np.concatenate((Tca_k1, Tsa_k, Tsa_k1, valve_k2), axis=1)
    tmp_y = np.array(df['supply_discharge_temp']).reshape(-1,1)[start+2: end+2]

    self.x_data = torch.tensor(tmp_x, dtype=torch.float32)
    self.y_data = torch.tensor(tmp_y, dtype=torch.float32)

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    inp  = self.x_data[idx]
    outp = self.y_data[idx]
    sample = {'inp':inp, 'outp':outp}
    return sample

In [5]:
# Early stopping function
def early_stop(list, min_epochs, patience):
    if(len(list) > min_epochs):
        if(np.max(list[-patience:]) < 1.00001*np.max(list[0: -patience])):
            return 1
    return 0

In [6]:
# train function
def train(net, train_ds, test_ds, lr=0.001, min_epochs=200, max_epochs=100000, patience=100, smooth=0):
    loss_func  = torch.nn.MSELoss()
    optimizer  = torch.optim.Adam(net.parameters(), lr=lr)

    R2_test = np.array([])
    train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=train_ds.y_data.shape[0], shuffle=True)
    for _ in range(0, max_epochs+1):
        net.train()
        for (_, batch) in enumerate(train_ldr):
            X = batch['inp']
            Y = batch['outp']

            optimizer.zero_grad()
            output = net(X)
            loss_val = loss_func(output, Y) + smooth*loss_func(output, X[:, 2].reshape(-1,1))
            loss_val.backward()
            optimizer.step()

        net.eval()
        R2_test = np.append(R2_test, eval(net, test_ds))
        
        if(early_stop(list = R2_test, min_epochs = min_epochs, patience = patience) == 1):
            break
    
    return R2_test

In [7]:
# main
for _n_train in [32, 64, 128]:
    for _smooth in [0, 0.001, 0.01]:
        for h in [16, 32, 64]:
            for _lr in [0.0001, 0.001, 0.01, 0.1]:

                # Create network
                device = torch.device("cpu")
                net = Net(h).to(device)

                # Create Dataset and DataLoader objects
                src_file = 'C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv'
                n_train  = _n_train
                train_ds = Data(src_file, start=0,       end=n_train)
                test_ds  = Data(src_file, start=n_train, end=1600)

                # train
                R2_test = train(net, train_ds, test_ds, lr=_lr, min_epochs=1000, max_epochs=100000, patience=500, smooth=_smooth)

                # results
                print('n train = %3d \t smooth = %6.4f \t layer size = %2d \t lr = %6.4f \t best_epoch = %5d \t best_R2 = %7.5f'
                    % (_n_train, _smooth, h, _lr, np.argmax(R2_test), np.max(R2_test)))

n train =  32 	 smooth = 0.0000 	 layer size = 16 	 lr = 0.0001 	 best_epoch = 64820 	 best_R2 = -0.40170
n train =  32 	 smooth = 0.0000 	 layer size = 16 	 lr = 0.0010 	 best_epoch =  7350 	 best_R2 = -0.05737
n train =  32 	 smooth = 0.0000 	 layer size = 16 	 lr = 0.0100 	 best_epoch = 17877 	 best_R2 = -0.39858
n train =  32 	 smooth = 0.0000 	 layer size = 16 	 lr = 0.1000 	 best_epoch =    77 	 best_R2 = -0.26306
n train =  32 	 smooth = 0.0000 	 layer size = 32 	 lr = 0.0001 	 best_epoch = 42366 	 best_R2 = 0.91003
n train =  32 	 smooth = 0.0000 	 layer size = 32 	 lr = 0.0010 	 best_epoch =  8956 	 best_R2 = 0.68940
n train =  32 	 smooth = 0.0000 	 layer size = 32 	 lr = 0.0100 	 best_epoch =  2721 	 best_R2 = 0.65613
n train =  32 	 smooth = 0.0000 	 layer size = 32 	 lr = 0.1000 	 best_epoch =    34 	 best_R2 = -0.00001
n train =  32 	 smooth = 0.0000 	 layer size = 64 	 lr = 0.0001 	 best_epoch = 23582 	 best_R2 = 0.76518
n train =  32 	 smooth = 0.0000 	 layer size = 64 